In [1]:
import symforce
symforce.set_epsilon_to_symbol()
from symforce.values import Values
import numpy as np
import cv2 as cv
import symforce.symbolic as sf

In [2]:
from symforce.opt.factor import Factor
from symforce.opt.optimizer import Optimizer

In [3]:
# make the sampled data: 
ar = 1.0
br = 2.0
cr = 1.0
ae = 2.0
be = -1.0
ce = 5.0
N = 100
w_sigma = 1.0
inv_sigma = 1.0 / w_sigma

rng = np.random.default_rng()

x_data = np.linspace(0, 1, N)
y_data = np.exp(ar * x_data * x_data + br * x_data + cr) + rng.normal(0, w_sigma, N)

In [4]:
def error(abc: sf.V3, x: sf.Scalar, y: sf.Scalar) -> sf.V1:
    return sf.V1(y - sf.exp(abc[0]*x*x + abc[1]*x + abc[2]))


In [5]:
initial_values = Values(
    abc = sf.V3([ae, be, ce]),
    xdata = x_data.tolist(),
    ydata = y_data.tolist(),
    epsilon = sf.numeric_epsilon,
)

In [6]:
factors = []
for i in range(len(x_data)):
    factors.append(Factor(
        residual=error,
        keys=["abc", f"xdata[{i}]", f"ydata[{i}]"],
    ))

In [7]:
optimizer = Optimizer(
    factors=factors,
    optimized_keys=["abc"],
    debug_stats=True,
    params=Optimizer.Params(iterations=1000)
)
result = optimizer.optimize(initial_values)
result.optimized_values.get('abc')

array([1.04825736, 1.91792162, 1.03197633])

In [8]:
a, b, c = result.optimized_values.get('abc')
y_cal = np.exp(a * x_data * x_data + b * x_data + c)
y_gt = np.exp(ar * x_data * x_data + br * x_data + cr)

In [9]:
import matplotlib.pyplot as plt
%matplotlib qt
plt.plot(x_data, y_data, '.')
plt.plot(x_data, y_cal)
plt.plot(x_data, y_gt)
plt.show()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""
